# Advanced Bayes Search CV Example

This is a more advanced example of how the `BayesSearchCV` class can be applied - it's recommended that you first read through the simpler `bayes_search_cv_example`.

The `BayesSearchCV` class is used to search for the set of hyperparameters that produce the best decision engine performance for a given Iguanas Pipeline, whilst also reducing the likelihood of overfitting.

The process is as follows:

* Generate k-fold stratified cross validation datasets. 
* For each of the training and validation datasets:
    * Fit the pipeline on the training set using a set of parameters chosen by the Bayesian Optimiser from a given set of ranges.
    * Apply the pipeline to the validation set to return a prediction.
    * Use the provided `scorer` to calculate the score of the prediction.
* Return the parameter set which generated the highest mean overall score across the validation datasets.

In this example, we'll consider the following more advanced workflow (compared to the standard `bayes_search_cv_example` notebook), which considers the generation of a Rules-Based System for a credit card fraud transaction use case:

<center><img src="images/complex_example.png"/></center>

Here, we have a fraud detection use case, and we're aiming to create two distinct rule sets - one for flagging fraudulent behaviour (which we'll refer to as our **Reject** rule set); one for flagging good behaviour (which we'll refer to as our **Approve** rule set). Each of these rule sets will be comprised of a generated rule set and an existing rule set. We'll optimise and filter these two rule sets separately, then combine and feed them into the decision engine optimiser. **Note:** we optimise the generated rules as they'll be created using the `RuleGeneratorDT` class, which generates rules from the branches of decision trees - these split based on gini or entropy - so we can further optimise them for a specific metric. 

**The decision engine will have the following constraint:** for a given transaction, if any approve rules fire it will be approved; else, if any reject rules fire it will be rejected; else, it will be approved.

We'll use the `BayesSearchCV` class to optimise the hyperparameters of the steps in this workflow, **ensuring that we maximise the revenue for our decision engine.**

---

## Import packages

In [1]:
from iguanas.rule_generation import RuleGeneratorDT
from iguanas.rule_selection import SimpleFilter, CorrelatedFilter, GreedyFilter, BayesSearchCV
from iguanas.metrics import FScore, Precision, Revenue, JaccardSimilarity
from iguanas.rbs import RBSOptimiser, RBSPipeline
from iguanas.correlation_reduction import AgglomerativeClusteringReducer
from iguanas.pipeline import LinearPipeline, ParallelPipeline
from iguanas.pipeline.class_accessor import ClassAccessor
from iguanas.space import UniformFloat, UniformInteger, Choice
from iguanas.rules import Rules
from iguanas.rule_optimisation import BayesianOptimiser

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from category_encoders.one_hot import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

## Read in data

Let's read in the [credit card fraud dataset](https://www.kaggle.com/mlg-ulb/creditcardfraud) from Kaggle.

**Note:** this data has been altered to include some null values in the `V1` column. This is to simulate unprocessed data (the dataset on Kaggle has been processed using PCA, so there are no null values). It has also been randomly sampled to 10% of its original number of records, to reduce the file size.

In [2]:
target_col = 'Class'
time_col = 'Time'
amt_col = 'Amount'
# Ready in data
df = pd.read_csv('dummy_data/creditcard.csv')
# Sort data by time ascending
df.sort_values(time_col, ascending=True)
# Create X and y dataframes
X = df.drop([target_col, time_col], axis=1)
y = df[target_col]

In [3]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.33,
    random_state=42
)

To calculate the **Revenue**, we need the monetary amount of each transaction - we'll use these later:

In [4]:
amts_train = X_train_raw[amt_col]
amts_test = X_test_raw[amt_col]

### Process data

Let's impute the null values with the mean:

In [5]:
imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(
    imputer.fit_transform(X_train_raw),
    columns=X_train_raw.columns,
    index=X_train_raw.index
)
X_test = pd.DataFrame(
    imputer.transform(X_test_raw),
    columns=X_test_raw.columns,
    index=X_test_raw.index
)

In [6]:
# Check nulls have been imputed
X_train.isna().sum().sum(), X_test.isna().sum().sum()

(0, 0)

### Existing rules

Let's also assume we have the following existing rules, stored in the standard Iguanas string format:

In [7]:
reject_rule_strings = {
    "ExistingReject1": "((X['V1']<0)|(X['V1'].isna()))&(X['V3']<1)",
    "ExistingReject2": "(X['V2']>3)",
}
approve_rule_strings = {
    "ExistingApprove1": "(X['V1']>0)&(X['V3']>1)",
    "ExistingApprove2": "(X['V2']<3)",
    "ExistingApprove3": "(X['V4']<3)"
}

We can create a `Rules` class for each of these:

In [8]:
reject_rules = Rules(rule_strings=reject_rule_strings)
approve_rules = Rules(rule_strings=approve_rule_strings)

Then convert them to the standard Iguanas lambda expression format (we'll need this for the optimisation step):

In [9]:
reject_rule_lambdas = reject_rules.as_rule_lambdas(
    as_numpy=False, 
    with_kwargs=True
)
approve_rule_lambdas = approve_rules.as_rule_lambdas(
    as_numpy=False, 
    with_kwargs=True
)

----

## Set up pipeline

Before we can apply the `BayesSearchCV` class, we need to set up our pipeline. To create the workflow shown at the beginning of the notebook, we must use a combination of `LinearPipeline` and `ParallelPipeline` classes as shown below:

![title](images/complex_example_setup.png)

Let's begin building the **Reject *LinearPipeline***.

### Reject *LinearPipeline*

Let's first instantiate the classes that we'll use in the pipeline:

In [10]:
# F1 Score
f1 = FScore(beta=1)
# Precision
p = Precision()
    
# Rule generation
reject_gen = RuleGeneratorDT(
    metric=f1.fit,
    n_total_conditions=2,
    tree_ensemble=RandomForestClassifier(
        n_estimators=10,
        random_state=0
    ),
    target_feat_corr_types='Infer',
    rule_name_prefix='Reject' # Set this so generated reject rules distinguishable from approve rules
)

# Rule optimisation (for generated rules)
reject_gen_opt = BayesianOptimiser(
    rule_lambdas=ClassAccessor(
        class_tag='reject_gen',
        class_attribute='rule_lambdas'
    ),
    lambda_kwargs=ClassAccessor(
        class_tag='reject_gen',
        class_attribute='lambda_kwargs'
    ),
    metric=f1.fit,
    n_iter=10
)

# Rule optimisation (for existing rules)
reject_opt = BayesianOptimiser(
    rule_lambdas=reject_rule_lambdas,
    lambda_kwargs=reject_rules.lambda_kwargs,
    metric=f1.fit,
    n_iter=10
)

# Rule filter (performance-based)
reject_sf = SimpleFilter(
    threshold=0.1, 
    operator='>=', 
    metric=f1.fit
)

# Rule filter (correlation-based)
js = JaccardSimilarity()
reject_cf = CorrelatedFilter(
    correlation_reduction_class=AgglomerativeClusteringReducer(
        threshold=0.9, 
        strategy='top_down', 
        similarity_function=js.fit, 
        metric=f1.fit
    ), 
    rules=ClassAccessor(
        class_tag='reject_gen',
        class_attribute='rules'
    )
)

Now we can create our **Reject Rule Generation *LinearPipeline***. Note that we pass the tag for the optimisation of the generated rules to the `use_init_data` parameter, so that the feature set is passed to the `BayesianOptimiser` class, rather than the output from the `RuleGeneratorDT`:

In [11]:
reject_gen_lp = LinearPipeline(
    steps = [
        ('reject_gen', reject_gen),
        ('reject_gen_opt', reject_gen_opt),
    ],
    use_init_data=['reject_gen_opt']
)

And then our **Reject *ParallelPipeline*** (noting that one of the steps in this pipeline is the **Reject Rule Generation *LinearPipeline*** created above):

In [12]:
reject_pp = ParallelPipeline(
    steps = [
        ('reject_gen_lp', reject_gen_lp),
        ('reject_opt', reject_opt),
    ]
)

And then finally, our **Reject *LinearPipeline***:

In [13]:
reject_lp = LinearPipeline(
    steps = [
        ('reject_pp', reject_pp),
        ('reject_sf', reject_sf),
        ('reject_cf', reject_cf)
    ]
)

Now we can do the same for the **Approve *LinearPipeline***:

### Approve *LinearPipeline*

Let's first instantiate the classes that we'll use in the pipeline:

In [14]:
# Rule generation
approve_gen = RuleGeneratorDT(
    metric=f1.fit,
    n_total_conditions=2,
    tree_ensemble=RandomForestClassifier(
        n_estimators=10,
        random_state=0
    ),
    target_feat_corr_types='Infer',
    rule_name_prefix='Approve' # Set this so generated reject rules distinguishable from approve rules
)

# Rule optimisation (for generated rules)
approve_gen_opt = BayesianOptimiser(
    rule_lambdas=ClassAccessor(
        class_tag='approve_gen',
        class_attribute='rule_lambdas'
    ),
    lambda_kwargs=ClassAccessor(
        class_tag='approve_gen',
        class_attribute='lambda_kwargs'
    ),
    metric=f1.fit,
    n_iter=10
)

# Rule optimisation (for existing rules)
approve_opt = BayesianOptimiser(
    rule_lambdas=approve_rule_lambdas,
    lambda_kwargs=approve_rules.lambda_kwargs,
    metric=f1.fit,
    n_iter=10
)

# Rule filter (performance-based)
approve_sf = SimpleFilter(
    threshold=0.1, 
    operator='>=', 
    metric=f1.fit
)

# Rule filter (correlation-based)
js = JaccardSimilarity()
approve_cf = CorrelatedFilter(
    correlation_reduction_class=AgglomerativeClusteringReducer(
        threshold=0.9, 
        strategy='top_down', 
        similarity_function=js.fit, 
        metric=f1.fit
    ),
    rules=ClassAccessor(
        class_tag='approve_gen',
        class_attribute='rules'
    )
)

Now we can create our **Approve Rule Generation *LinearPipeline***. Note that we pass the tag for the optimisation of the generated rules to the `use_init_data` parameter, so that the feature set is passed to the `BayesianOptimiser` class, rather than the output from the `RuleGeneratorDT`:

In [15]:
approve_gen_lp = LinearPipeline(
    steps = [
        ('approve_gen', approve_gen),
        ('approve_gen_opt', approve_gen_opt),
    ],
    use_init_data=['approve_gen_opt']
)

And then our **Approve *ParallelPipeline*** (noting that one of the steps in this pipeline is the **Approve Rule Generation *LinearPipeline*** created above):

In [16]:
approve_pp = ParallelPipeline(
    steps = [
        ('approve_gen_lp', approve_gen_lp),
        ('approve_opt', approve_opt),
    ]
)

And then finally, our **Approve *LinearPipeline***:

In [17]:
approve_lp = LinearPipeline(
    steps = [
        ('approve_pp', approve_pp),
        ('approve_sf', approve_sf),
        ('approve_cf', approve_cf)
    ]
)

Now we can move on to constructing the **Overall Pipelines:**

### Overall Pipelines

First, we'll construct our **Overall *ParallelPipeline*** using the **Reject *LinearPipeline*** and **Approve *LinearPipeline***:

In [18]:
overall_pp = ParallelPipeline(
    steps = [
        ('reject_lp', reject_lp),
        ('approve_lp', approve_lp)
    ]
)

Now we can instantiate the decision engine optimiser. Since we have a constraint on the decision engine (if any approve rules fire, approve the transaction; else if any reject rules fire, reject the transaction; else approve the transaction), we pass the rules remaining after the filtering stages to the relevant elements in the `config` parameter of the `RBSPipeline` class, using the `ClassAccessor` class:

In [19]:
# Decision engine optimisation metric
opt_metric = Revenue(
    y_type='Fraud',
    chargeback_multiplier=3
)

# Decision engine (to be optimised)
rbs_pipeline = RBSPipeline(
    config=[
        [
            0, ClassAccessor( # If any approve rules fire, approve
                class_tag='approve_cf', 
                class_attribute='rules_to_keep'
            ),
        ],
        [
            1, ClassAccessor( # Else if any reject rules fire, reject
                class_tag='reject_cf', 
                class_attribute='rules_to_keep'
            )
        ],        
    ],
    final_decision=0 # Else approve
)

# Decision engine optimiser
rbs_optimiser = RBSOptimiser(
    pipeline=rbs_pipeline,
    metric=opt_metric.fit,     
    rules=ClassAccessor(
        class_tag='overall_pp',
        class_attribute='rules'
    ),
    n_iter=10
)

Finally, we can instantiate our **Overall *LinearPipeline***:

In [20]:
overall_lp = LinearPipeline(
    steps=[
        ('overall_pp', overall_pp),
        ('rbs_optimiser', rbs_optimiser)
    ]
)

## Define the search space

Now we need to define the search space for each of the relevant parameters of our pipeline. **Note:** this example does not search across all hyperparameters - you should define your own search spaces based on your use case.

To do this, we create a dictionary, where each key corresponds to the tag used for the relevant pipeline step. Each value should be a dictionary of the parameters (keys) and their search spaces (values). Search spaces should be defined using the classes in the `iguanas.space` module:

In [21]:
# Define additional FScores
f0dot5 = FScore(beta=0.5)
f0dot25 = FScore(beta=0.25)

In [22]:
search_spaces = {
    'reject_gen': {
        'n_total_conditions': UniformInteger(2, 7),
    },
    'reject_gen_opt': {
        'metric': Choice([f0dot25.fit, f0dot5.fit, f1.fit]),
    },
    'reject_sf': {
        'threshold': UniformFloat(0, 1),
    },
    'reject_cf': {
        'correlation_reduction_class': Choice(
            [
                AgglomerativeClusteringReducer(
                    threshold=0.9, 
                    strategy='top_down', 
                    similarity_function=js.fit, 
                    metric=f1.fit                    
                ),
                AgglomerativeClusteringReducer(
                    threshold=0.95, 
                    strategy='top_down', 
                    similarity_function=js.fit, 
                    metric=f1.fit                    
                )
            ]
        )
    },    
    'approve_gen': {
        'n_total_conditions': UniformInteger(2, 7),
    },
    'approve_gen_opt': {
        'metric': Choice([f0dot25.fit, f0dot5.fit, f1.fit]),
    },
    'approve_sf': {
        'threshold': UniformFloat(0, 1),
    },
    'approve_cf': {
        'correlation_reduction_class': Choice(
            [
                AgglomerativeClusteringReducer(
                    threshold=0.9, 
                    strategy='top_down', 
                    similarity_function=js.fit, 
                    metric=f1.fit                    
                ),
                AgglomerativeClusteringReducer(
                    threshold=0.95, 
                    strategy='top_down', 
                    similarity_function=js.fit, 
                    metric=f1.fit                    
                )
            ]
        )
    }
}

## Optimise the pipeline hyperparameters

Now that we have our pipeline and search spaces defined, we can instantiate the `BayesSearchCV` class. We'll split our data into 3 cross-validation datasets and try 10 different parameter sets.

**Note:** since we're using the `Revenue` as the scoring metric for the `BayesSearchCV` class, we need to set the `sample_weight_in_val` parameter to `True`. This ensures that the `sample_weight` passed to the final step in the pipeline is used when applying the `metric` function to the prediction of each validation set (for `Revenue`, the `sample_weight` corresponds to the monetary amount of each transaction, which is required).

In [23]:
bs = BayesSearchCV(
    pipeline=overall_lp, 
    search_spaces=search_spaces, 
    metric=opt_metric.fit, # Use the same metric as the RBSOptimiser
    cv=3, 
    n_iter=10,
    num_cores=3,
    error_score=0,
    verbose=1,
    sample_weight_in_val=True # Set to True
)

Finally, we can run the `fit` method to optimise the hyperparameters of the pipeline. 

**Note the following:** 

* The existing rules contain conditions that rely on unprocessed data (in this case, there are conditions that check for nulls). So for the rule optimisation steps, we must use the unprocessed training data `X_train_raw`; for the rule generation steps, we must use the processed training data `X_train`.
* Since we're generating and optimising rules that flag both positive and negative cases (i.e. reject and approve rules in this example), we need to specify what the target is in each case. For the reject rules, we can just use `y_train`, however for the approve rules, we need to flip `y_train` (so that the rule generator and rule optimisers target the negative cases).
* We need the `amts_train` to be passed to the `sample_weight` parameter of the `RBSOptimiser`, as we're optimising the decision engine for the `Revenue`.

In [24]:
bs.fit(
    X={
        'reject_lp': X_train, # Use processed features for rule generation
        'reject_opt': X_train_raw, # Use raw features for optimising existing rules
        'approve_lp': X_train, # Use processed features for rule generation
        'approve_opt': X_train_raw # Use raw features for optimising existing rules
    }, 
    y={
        'reject_lp': y_train, # Use target for Reject LinearPipeline
        'approve_lp': 1-y_train, # Flip target for Approve LinearPipeline
        'rbs_optimiser': y_train # Use target for RBSOptimiser
    },
    sample_weight={
        'reject_lp': None, # No sample_weight for Reject LinearPipeline
        'approve_lp': None, # No sample_weight for Approve LinearPipeline
        'rbs_optimiser': amts_train # sample_weight for RBSOptimiser
    }
)

--- Optimising pipeline parameters ---
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:13<00:00,  7.37s/trial, best loss: -555025.6433333332]
--- Refitting on entire dataset with best pipeline ---


### Outputs

The `fit` method doesn't return anything. See the `Attributes` section in the class docstring for a description of each attribute generated:

In [25]:
bs.best_score

555025.6433333332

In [26]:
bs.best_params

{'approve_cf': {'correlation_reduction_class': AgglomerativeClusteringReducer(threshold=0.9, strategy=top_down, similarity_function=<bound method JaccardSimilarity.fit of JaccardSimilarity>, metric=<bound method FScore.fit of FScore with beta=1>, print_clustermap=False)},
 'approve_gen': {'n_total_conditions': 7.0},
 'approve_gen_opt': {'metric': <bound method FScore.fit of FScore with beta=0.5>},
 'approve_sf': {'threshold': 0.43381170823234194},
 'reject_cf': {'correlation_reduction_class': AgglomerativeClusteringReducer(threshold=0.95, strategy=top_down, similarity_function=<bound method JaccardSimilarity.fit of JaccardSimilarity>, metric=<bound method FScore.fit of FScore with beta=1>, print_clustermap=False)},
 'reject_gen': {'n_total_conditions': 5.0},
 'reject_gen_opt': {'metric': <bound method FScore.fit of FScore with beta=0.25>},
 'reject_sf': {'threshold': 0.5309641649521473}}

In [27]:
bs.best_index

1

In [28]:
bs.cv_results.head()

,Params,approve_cf__correlation_reduction_class,approve_gen__n_total_conditions,approve_gen_opt__metric,approve_sf__threshold,reject_cf__correlation_reduction_class,reject_gen__n_total_conditions,reject_gen_opt__metric,reject_sf__threshold,FoldIdx,Scores,MeanScore,StdDevScore
1,{'approve_cf': {'correlation_reduction_class':...,"AgglomerativeClusteringReducer(threshold=0.9, ...",7.0,<bound method FScore.fit of FScore with beta=0.5>,0.433812,"AgglomerativeClusteringReducer(threshold=0.95,...",5.0,<bound method FScore.fit of FScore with beta=0...,0.530964,"[0, 1, 2]","[572829.7, 569910.5599999999, 522336.67000000004]",555025.643333,23145.295818
3,{'approve_cf': {'correlation_reduction_class':...,"AgglomerativeClusteringReducer(threshold=0.95,...",2.0,<bound method FScore.fit of FScore with beta=0...,0.457207,"AgglomerativeClusteringReducer(threshold=0.9, ...",4.0,<bound method FScore.fit of FScore with beta=0...,0.528692,"[0, 1, 2]","[573229.7, 555044.7199999999, 535382.05]",554552.156667,15455.163465
0,{'approve_cf': {'correlation_reduction_class':...,"AgglomerativeClusteringReducer(threshold=0.95,...",3.0,<bound method FScore.fit of FScore with beta=0.5>,0.195596,"AgglomerativeClusteringReducer(threshold=0.9, ...",4.0,<bound method FScore.fit of FScore with beta=1>,0.486047,"[0, 1, 2]","[572817.9400000001, 555595.08, 524511.97]",550974.996667,19989.589989
8,{'approve_cf': {'correlation_reduction_class':...,"AgglomerativeClusteringReducer(threshold=0.95,...",7.0,<bound method FScore.fit of FScore with beta=0...,0.027295,"AgglomerativeClusteringReducer(threshold=0.95,...",3.0,<bound method FScore.fit of FScore with beta=1>,0.510489,"[0, 1, 2]","[572121.0199999999, 538455.7999999999, 535382.05]",548652.956667,16641.804434
9,{'approve_cf': {'correlation_reduction_class':...,"AgglomerativeClusteringReducer(threshold=0.95,...",6.0,<bound method FScore.fit of FScore with beta=0...,0.933435,"AgglomerativeClusteringReducer(threshold=0.95,...",5.0,<bound method FScore.fit of FScore with beta=1>,0.092054,"[0, 1, 2]","[573019.7, 541266.7200000001, 528778.51]",547688.310000,18623.432135


To see the final optimised decision engine configuration and rule set, we first return the parameters of the trained pipeline (stored in the attribute `pipeline_`):

In [29]:
pipeline_params = bs.pipeline_.get_params()

Then, to see the final optimised decision engine configuration, we filter to the `config` parameter of the `rbs_optimiser` step:

In [30]:
final_config = pipeline_params['rbs_optimiser']['config']
final_config

[[0, ['Approve_44', 'Approve_73', 'Approve_40']],
 [1,
  ['Reject_15',
   'Reject_18',
   'Reject_4',
   'Reject_17',
   'Reject_21',
   'Reject_2',
   'Reject_29',
   'Reject_19']]]

This shows us which rules should be used for the approval step (decision `0`) and which rules should be used for the rejection step (decision `1`).

To see the logic of our final set of rules, we filter to the `rules` parameter of the `rbs_optimiser` step:

In [31]:
final_rules = bs.pipeline_.get_params()['rbs_optimiser']['rules']

Then extract the `rule_strings` attribute:

In [32]:
final_rules.rule_strings

{'Reject_2': "(X['V1']<=1.57711)&(X['V14']<=-5.9433)&(X['V26']>-0.27216)",
 'Reject_4': "(X['V10']<=-2.70569)&(X['V11']>3.26894)&(X['V19']>-3.50737)",
 'Reject_15': "(X['V12']<=-2.1202)&(X['V14']<=-4.64098)",
 'Reject_17': "(X['V12']<=-4.43659)",
 'Reject_18': "(X['V12']<=-4.43715)",
 'Reject_19': "(X['V12']<=-4.59928)",
 'Reject_21': "(X['V14']<=-11.33548)&(X['V17']<=-2.69333)&(X['V27']>1.1957)",
 'Reject_29': "(X['V17']<=-2.18248)&(X['V27']>-4.29354)",
 'Approve_40': "(X['V11']<=3.35091)&(X['V12']>-3.67614)&(X['V13']>0.43179)&(X['V17']>-2.69333)",
 'Approve_44': "(X['V11']<=3.35091)&(X['V12']>-4.59928)&(X['V16']>-2.1745)&(X['V17']>-2.69333)&(X['V19']<=-1.48536)",
 'Approve_73': "(X['V17']>-2.18248)&(X['V19']<=-3.52538)"}

## Apply the optimised pipeline

We can apply our optimised pipeline to a new data set and make a prediction using the `predict` method:

In [33]:
y_pred_test = bs.predict(X_test)

### Outputs

The `predict` method returns the prediction generated by class in the final step of the pipeline - in this case, the `RBSOptimiser`:

In [34]:
y_pred_test

17855    0
23775    0
18629    0
12843    0
18084    0
        ..
2223     0
10168    0
23823    0
16583    0
22478    0
Length: 9399, dtype: int64

We can now calculate the **Revenue** of our optimised pipeline using the test data:

In [35]:
rev_opt = opt_metric.fit(
    y_preds=y_pred_test,
    y_true=y_test,
    sample_weight=amts_test
)

Comparing this to our original, unoptimised pipeline:

In [36]:
overall_lp.fit(
    X={
        'reject_gen_lp': X_train,
        'reject_opt': X_train_raw,
        'approve_gen_lp': X_train,
        'approve_opt': X_train_raw    
    }, 
    y={
        'reject_lp': y_train,
        'approve_lp': 1-y_train,
        'rbs_optimiser': y_train
    },
    sample_weight={
        'reject_lp': None,
        'approve_lp': None,
        'rbs_optimiser': y_train        
    }
)

In [37]:
y_pred_test_init = overall_lp.predict(X_test)

In [38]:
rev_init = opt_metric.fit(
    y_preds=y_pred_test_init,
    y_true=y_test,
    sample_weight=amts_test
)

In [39]:
print(f'Revenue of original, unoptimised pipeline: ${round(rev_init)}')
print(f'Revenue of optimised pipeline: ${round(rev_opt)}')
print(f'Absolute improvement in Revenue: ${round(rev_opt-rev_init)}')
print(f'Percentage improvement in Revenue: {round(100*(rev_opt-rev_init)/rev_init, 2)}%')

Revenue of original, unoptimised pipeline: $775698
Revenue of optimised pipeline: $856076
Absolute improvement in Revenue: $80379
Percentage improvement in Revenue: 10.36%


/Users/jlaidler/Documents/Iguanas/iguanas/pipeline/parallel_pipeline.py:151: NoRulesWarning: No rules remain in step `reject_lp` as it raised the following error: "`X` has been reduced to zero columns after the `reject_sf` step in the pipeline."
  warnings.warn(
/Users/jlaidler/Documents/Iguanas/iguanas/rule_selection/bayes_search_cv.py:518: NoRulesWarning: No rules remaining for: Pipeline parameter set = {'approve_cf': {'correlation_reduction_class': AgglomerativeClusteringReducer(threshold=0.9, strategy=top_down, similarity_function=<bound method JaccardSimilarity.fit of JaccardSimilarity>, metric=<bound method FScore.fit of FScore with beta=1>, print_clustermap=False)}, 'approve_gen': {'n_total_conditions': 3.0}, 'approve_gen_opt': {'metric': <bound method FScore.fit of FScore with beta=1>}, 'approve_sf': {'threshold': 0.48617287843841694}, 'reject_cf': {'correlation_reduction_class': AgglomerativeClusteringReducer(threshold=0.95, strategy=top_down, similarity_function=<bound method

---